# LHE pdf-variations and qcd weights

In [6]:
import os, sys, pickle, copy
import datetime
sys.path.append(os.getcwd() + "/../")
from prep_ana_II import *
sys.path.append(os.getcwd() + "/../tools/PdfQcd_unc_tools")
from pdf_qcd_tools import get_pdf_weights, calc_unc, calc_qcd_unc, calc_qcd_unc_min_max
sys.path.append(os.getcwd() + "/../scripts/uncertainties_dir/")
from jet_scale import jec_setup, jet_scale_shift_flat, pseudo_data_yield_sum
from cross_section_calc import calc_cross_stuff, cross_calc, stat_unc_calc, normalization_unc_calc
import warnings
from tools import x_per_jet_bin, yield_string
import pdf_qcd_tools
warnings.filterwarnings("ignore")

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 

In [3]:
def calc_lhe_weight(df, element):
  weights = np.array(get_pdf_weights(list(df.lhe_weight_string.values), element))
  return weights


def calc_rmsd(some_dic, nominal, jet_bin = 'tot'):
  result = 0
  it = 0.
  for pdf in some_dic:
    print pdf, some_dic[pdf][jet_bin]
    it += 1.
    result += (nominal - some_dic[pdf][jet_bin]) **2.

  result = (result / it)**.5
  return result

In [4]:
jer_obj = jec_setup(unc="lhe")
jet_scale_shift_flat(jer_obj.df, jet_pt_shift=1., rf=jer_obj.rfs)#?Doesn't work. FIX tools.py
df = pre_cuts(jer_obj.df, diff_charge= False)
df_ww = jer_obj.df_ww
warnings.filterwarnings("ignore")

Warning in <TStreamerInfo::BuildCheck>: 
   The StreamerInfo of class string read from file /home/gunter/WW_analysis/data_alpha/ww_complete.root
   has the same version (=2) as the active class but a different checksum.
   You should update the version to ClassDef(string,3).
   Do not try to write objects with the current class definition,
   the files will not be readable.

Warning in <TStreamerInfo::CompareContent>: The following data member of
the in-memory layout version 2 of class 'string' is missing from 
the on-file layout version 2:
   string::_Alloc_hider _M_dataplus; //
Warning in <TStreamerInfo::CompareContent>: The following data member of
the in-memory layout version 2 of class 'string' is missing from 
the on-file layout version 2:
   unsigned long _M_string_length; //
Warning in <TStreamerInfo::CompareContent>: The following data member of
the in-memory layout version 2 of class 'string' is missing from 
the on-file layout version 2:
   string::(anonymous) ; //


# QCD scale variations

In [ ]:
#Create copy of weights for future use
static_weights_df = copy.copy(df.weight)
static_weights_ww = copy.copy(jer_obj.df_ww.weight)


pseudo = {}
x_per_jet_bin(pseudo, df, jer_obj.df_da, jer_obj.df_ww, pseudo_data_yield_sum)

df_process_yields = process_yields(df[(df.pred_fTT_WW > .6) & (df.pred_fDY_WW > .9)])
print "orig, top", df_process_yields[df_process_yields.Process == "Top"]
nominal_dic = {}
x_per_jet_bin(nominal_dic, df, jer_obj.df_da, jer_obj.df_ww, cross_calc, args={"scales": scales, "fiducial":True, "pseudo":pseudo})
orig_values = df[(df.process == process) & (df.pred_fTT_WW > .6) & (df.pred_fDY_WW > .9)].weight.mean()

##
impacts = []
for process in ["Top"]:#df.process.unique():
    print "process", process
    post_dic = {}
    diff_cross_section = []

    temp_weights = np.array(calc_qcd_unc(list(df[df.process == process].lhe_weight_string)))
    temp_weights = np.array(calc_qcd_unc_min_max(list(df[df.process == process].lhe_weight_string), 0))
    temp_weights[temp_weights > 20] = temp_weights[temp_weights < 20].mean()
    df.weight.values[df.process == process] = df.weight.values[df.process == process] * temp_weights

    if process == "WW":
        temp_weights_ww = np.array(calc_qcd_unc(list(df_ww[df_ww.process == process].lhe_weight_string), element))
        temp_weights_ww[temp_weights_ww > 20] = temp_weights_ww[temp_weights_ww < 20].mean()
        jer_obj.df_ww["weight"] = jer_obj.df_ww.weight * temp_weights_ww

    plt.hist(temp_weights[(df[df.process == "Top"].pred_fTT_WW > .6) & (df[df.process == "Top"].pred_fDY_WW > .9)], bins=50, range=(0, 2))
    plt.yscale("log")
    plt.title("Max diff")
    plt.show()

    x_per_jet_bin(post_dic, df, jer_obj.df_da, jer_obj.df_ww, cross_calc, args={"scales": scales, "fiducial":True, "pseudo":pseudo})
    impacts.append((nominal_dic["tot"] - post_dic["tot"]) / nominal_dic["tot"])
    
    max_diff_values = df[(df.process == process) & (df.pred_fTT_WW > .6) & (df.pred_fDY_WW > .9)].weight.mean()
    temp_weights_mean = temp_weights[(df[df.process == "Top"].pred_fTT_WW > .6) & (df[df.process == "Top"].pred_fDY_WW > .9)].mean()
    print "Max Diff", temp_weights_mean, nominal_dic["tot"],\
    (max_diff_values - orig_values) / orig_values,\
    (nominal_dic["tot"] - post_dic["tot"]) / nominal_dic["tot"]
    
    df_process_yields = process_yields(df[(df.pred_fTT_WW > .6) & (df.pred_fDY_WW > .9)])
    print df_process_yields[df_process_yields.Process == "Top"]
    
    print "mean\tmax\tmin\tdiff in Xs"
    #"""
    for element in range(1, 9):
      df["weight"]    = 1 * static_weights_df[:]
      df_ww["weight"] = 1 * static_weights_ww[:]

      if element == 5:
        continue
      if element == 7:
        continue
      temp_weights = np.array(get_pdf_weights(list(df[df.process == process].lhe_weight_string), element))
      temp_weights[temp_weights > 20] = temp_weights[temp_weights < 20].mean()
      pre_weight_mean = df.weight.values[(df.process == process) & (df.pred_fTT_WW > .6) & (df.pred_fDY_WW > .9) ].mean()
      pre_weight_mean_ = df.weight.values[(df.process == process) ].mean()
      df.weight.values[df.process == process] = df.weight.values[df.process == process] * temp_weights
      post_weight_mean = df.weight.values[(df.process == process) & (df.pred_fTT_WW > .6) & (df.pred_fDY_WW > .9)].mean()
      post_weight_mean_ = df.weight.values[(df.process == process) ].mean()

      if process == "WW":
        temp_weights_ww = np.array(get_pdf_weights(list(df_ww[df_ww.process == process].lhe_weight_string), element))
        temp_weights_ww[temp_weights_ww > 20] = temp_weights_ww[temp_weights_ww < 20].mean()
        jer_obj.df_ww["weight"] = jer_obj.df_ww.weight * temp_weights_ww

      post_dic = {}
      x_per_jet_bin(post_dic, df, jer_obj.df_da, jer_obj.df_ww, cross_calc, args={"scales": scales, "fiducial":True, "pseudo":pseudo})

      print "element", element,\
        temp_weights.mean(),\
        temp_weights.max(),\
        (nominal_dic["tot"] - post_dic["tot"]) / nominal_dic["tot"],\
        (pre_weight_mean_ - post_weight_mean_) / pre_weight_mean

      df_process_yields = process_yields(df[(df.pred_fTT_WW > .6) & (df.pred_fDY_WW > .9)])
      #print df_process_yields[df_process_yields.Process == "Top"]

      plt.hist(temp_weights[(df[df.process == "Top"].pred_fTT_WW > .6) & (df[df.process == "Top"].pred_fDY_WW > .9) ], bins=50, range=(0,5))
      plt.yscale("log")
      plt.title("element " + str(element))
      plt.show()
    #"""


    df["weight"]    = 1 * static_weights_df[:]
    df_ww["weight"] = 1 * static_weights_ww[:]


sumquad_impacts = sum([impact**2 for impact in impacts])**.5
print "FIN", sumquad_impacts


orig, top    Diff Flavor Process  Same Flavor
3         5611     Top         1056
process Top
